In [1]:
from __future__ import print_function
import numpy as np
import torch
import torch.autograd as ag
import mnist_reader
import MNISTtools
import matplotlib.pyplot as plt
import time
from tqdm import tqdm_notebook
from sklearn.metrics import confusion_matrix
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import data_split
import gc

In [2]:
# Can also add data augmentation transforms here
train_transform = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(), transforms.Lambda(lambda x: torch.cat([x,x,x],0)) ])

# transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))   #transforms.ToPILImage(),
test_transform = transforms.Compose([ transforms.Resize(224), 
                                     transforms.ToTensor(), transforms.Lambda(lambda x: torch.cat([x,x,x],0)) ])

In [3]:
# print('Downloading/Checking for data......')
trainset = torchvision.datasets.FashionMNIST(root='data/downloads', train=True,
                                        download=True, transform=train_transform)       # download=True for the 1st time
testset = torchvision.datasets.FashionMNIST(root='data/downloads', train=False,
                                        download=True, transform=test_transform)        # download=True for the 1st time
train, validation = data_split.train_valid_split(trainset)    # separates 10% for validation

In [4]:
trainloader = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validation, batch_size=10, shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False, num_workers=2)

In [5]:
print('# of batches in training: ',len(trainloader), ',   Total train data: ',len(trainloader)*5)
print('# of batches in test: ',len(testloader), ',   Total test data: ',len(testloader)*5)
img, label = train[100]
print('image size: ', img.size())
# plt.imshow(img[0,:,:])
# print(type(label))

# of batches in training:  5400 ,   Total train data:  27000
# of batches in test:  1000 ,   Total test data:  5000
image size:  torch.Size([3, 224, 224])


# Loading the VGG16 trained on imagenet

In [6]:
vgg16 = torchvision.models.vgg16_bn(pretrained='imagenet')
# print(vgg16.classifier[6].out_features)        # 1000, as it was trained for 1000 classes

#--------------------------------------------------------------------------------------
# REMEMBER: vgg16.features --> convolutional layers, vgg16.classifier --> FC layers
#--------------------------------------------------------------------------------------

# freeze all parameters in covolutional layers
for parameter in vgg16.features.parameters():
    parameter.require_grad = False

in_ftrs = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1]       # Removing last layer to add out 10 units layer
features.extend([nn.Linear(in_ftrs, 10)])               # adding out layer with 10 units
vgg16.classifier = nn.Sequential(*features)             # replacing it with the model with new last layer

In [7]:
# print(vgg16)  # prints the architecture

In [8]:
vgg16=vgg16.cuda()

In [9]:
def test(vgg16):
    correct = 0
    total = 0
    acc_test = 0.0
    
    vgg16.train(False)
    vgg16.eval()    
    for i,data in enumerate(testloader):
        images, labels = data

        images = images.cuda()
        labels = labels.cuda()

        images = ag.Variable(images, volatile=True)
        labels = ag.Variable(labels, volatile=True)

        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        acc_test += torch.sum(predicted == labels.data)
        del outputs, predicted
        torch.cuda.empty_cache()
    #     gc.collect()

    print(acc_test/len(testset))
    # print('Test Accuracy of the model on the 10000 test images: %.4f %%' % (100 * correct / total))

In [10]:
test(vgg16)

0.0606


# Training the model

In [10]:
vgg16.train(True)
# vgg16.train()
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [11]:
start=time.time()
for epoch in range(3):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):       # 0 is just to start i from 0
        # get the inputs
        inputs, labels = data
        
        inputs = ag.Variable(inputs, requires_grad = False)
        labels = ag.Variable(labels, requires_grad = False)
        
        # transformations
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg16(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss[0].data.cpu().numpy()[0]
        # print(type(loss[0].data.cpu().numpy()[0]))
        if i==50:
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 50))
        if i==100:
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 100))
        if i==200:
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 200))
        if i==500:
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 500))
        
        if i % 500 == 499:    # print every 500 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0
        del inputs, labels, loss, outputs
        torch.cuda.empty_cache()
    print('epoch: ', epoch)
    print('Total time taken in training (secs): ',time.time()-start)
print('Total time taken in training (secs): ',time.time()-start)
print('Finished Training')

[1,    51] loss: 1.756
[1,   101] loss: 1.287
[1,   201] loss: 0.971
[1,   500] loss: 0.668
[1,   501] loss: 0.000
[1,  1000] loss: 0.370
[1,  1500] loss: 0.338
[1,  2000] loss: 0.320
[1,  2500] loss: 0.267
[1,  3000] loss: 0.258
[1,  3500] loss: 0.254
[1,  4000] loss: 0.236
[1,  4500] loss: 0.237
[1,  5000] loss: 0.225
epoch:  0
Total time taken in training (secs):  1110.70010591
[2,    51] loss: 0.230
[2,   101] loss: 0.226
[2,   201] loss: 0.204
[2,   500] loss: 0.199
[2,   501] loss: 0.000
[2,  1000] loss: 0.192
[2,  1500] loss: 0.189
[2,  2000] loss: 0.181
[2,  2500] loss: 0.176
[2,  3000] loss: 0.192
[2,  3500] loss: 0.172
[2,  4000] loss: 0.185
[2,  4500] loss: 0.179
[2,  5000] loss: 0.175
epoch:  1
Total time taken in training (secs):  2224.13101792
[3,    51] loss: 0.147
[3,   101] loss: 0.162
[3,   201] loss: 0.149
[3,   500] loss: 0.133
[3,   501] loss: 0.000
[3,  1000] loss: 0.151
[3,  1500] loss: 0.146
[3,  2000] loss: 0.143
[3,  2500] loss: 0.135
[3,  3000] loss: 0.149
[3

In [ ]:
torch.cuda.empty_cache()

In [31]:
# test(vgg16)    # 1 epoch

0.9162


In [12]:
test(vgg16)

0.9311
